In [1]:
from django_bootstrap import bootstrap
bootstrap()

from django.core import serializers
import json

In [2]:
major_cupt_code_strs="""10020118620101A-A
10020118620101A-B
10020326610502B-
10020326620101A-A
10020326620101A-B
10020326620101B-A
10020326620101B-B"""

In [3]:
from criteria.models import *

In [4]:
project_ids = [27,28]

program_major_codes = [s.split("-") for s in major_cupt_code_strs.split("\n")]
print(program_major_codes)

[['10020118620101A', 'A'], ['10020118620101A', 'B'], ['10020326610502B', ''], ['10020326620101A', 'A'], ['10020326620101A', 'B'], ['10020326620101B', 'A'], ['10020326620101B', 'B']]


In [5]:
criterias = {}

for p,m in program_major_codes:
    cupt_code = MajorCuptCode.objects.filter(program_code=p, major_code=m)[0]
    data1 = serializers.serialize('json',[cupt_code])
    data2 = json.loads(data1)[0]['fields']
    
    criterias[(p,m)] = {
        'data': {
            'major_cupt_code': data2,
            'criterias': {},
        },
        'objects': {
            'cupt_code': cupt_code,
            'criterias': {},
        }
    }
    
print(criterias)

{('10020118620101A', 'A'): {'data': {'major_cupt_code': {'program_code': '10020118620101A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 30, 'major_code': 'A', 'title': 'บธ.บ. สาขาวิชาการจัดการอุตสาหกรรมการบริการ', 'major_title': 'วิชาเอกการจัดการโรงแรม', 'component_weight_type': ''}, 'criterias': {}}, 'objects': {'cupt_code': <MajorCuptCode: บธ.บ. สาขาวิชาการจัดการอุตสาหกรรมการบริการ วิชาเอกการจัดการโรงแรม (ภาษาไทย ปกติ)>, 'criterias': {}}}, ('10020118620101A', 'B'): {'data': {'major_cupt_code': {'program_code': '10020118620101A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 30, 'major_code': 'B', 'title': 'บธ.บ. สาขาวิชาการจัดการอุตสาหกรรมการบริการ', 'major_title': 'วิชาเอกการจัดการการท่องเที่ยวร่วมสมัย', 'component_weight_type': ''}, 'criterias': {}}, 'objects': {'cupt_code': <MajorCuptCode: บธ.บ. สาขาวิชาการจัดการอุตสาหกรรมการบริการ วิชาเอกการจัดการการท่องเที่ยวร่วมสมัย (ภาษาไทย ปกติ)>, 'criterias': {}}}, ('10020326610502B', ''): {'da

In [6]:
cupt_code

<MajorCuptCode: บธ.บ. สาขาวิชาการจัดการอุตสาหกรรมการบริการ วิชาเอกการจัดการการการท่องเที่ยวร่วมสมัย (ภาษาไทย พิเศษ)>

In [7]:
def serialize_admission_criteria(criteria):
    data1 = serializers.serialize('json',[criteria])
    data2 = json.loads(data1)[0]
    
    score_data1 = serializers.serialize('json', criteria.scorecriteria_set.all())
    score_data2 = json.loads(score_data1)
    
    result = {
        'admission_criteria': data2['fields'],
        'score_criterias': [d['fields'] for d in score_data2],
    }

    print(result)
    return result

In [8]:
for p,m in criterias:
    k = (p,m)
    cupt_code = criterias[k]['objects']['cupt_code']
    print(cupt_code)
    curriculum_majors = CurriculumMajor.objects.filter(cupt_code=cupt_code)
    for m in curriculum_majors:
        if m.admission_project_id not in project_ids:
            continue

        print(m.admission_project_id)
        major_admission_criterias = CurriculumMajorAdmissionCriteria.objects.filter(curriculum_major=m)
        
        criterias[k]['objects']['criterias'][m.admission_project_id] = [
            {'slots':cmac.slots,
             'admission_criteria': cmac.admission_criteria}
            for cmac in major_admission_criterias
            if cmac.admission_criteria.is_deleted == False
        ]
        criterias[k]['data']['criterias'][m.admission_project_id] = [
            {'slots':cmac.slots,
             'admission_criteria': serialize_admission_criteria(cmac.admission_criteria)}
            for cmac in major_admission_criterias
            if cmac.admission_criteria.is_deleted == False
        ]
        
    print('------------------')
    print()

บธ.บ. สาขาวิชาการจัดการอุตสาหกรรมการบริการ วิชาเอกการจัดการโรงแรม (ภาษาไทย ปกติ)
27
{'admission_criteria': {'admission_project': 27, 'faculty': 30, 'version': 2, 'is_deleted': False, 'created_at': '2021-02-01T17:01:31.999', 'created_by': 'regis', 'additional_description': '', 'additional_condition': '', 'curriculum_majors_json': '[{"curriculum_major_id": 2012, "slots": 15, "major_cupt_code_id": 278, "program_code": "10020118620101A", "program_type_code": "A", "major_code": "A"}]'}, 'score_criterias': [{'admission_criteria': 7452, 'version': 2, 'primary_order': 1, 'secondary_order': 0, 'criteria_type': 'required', 'score_type': 'OTHER', 'value': '16.0000', 'unit': '', 'description': 'O-NET (03) ภาษาอังกฤษ', 'parent': None, 'relation': None}, {'admission_criteria': 7452, 'version': 2, 'primary_order': 1, 'secondary_order': 0, 'criteria_type': 'scoring', 'score_type': 'OTHER', 'value': '1.0000', 'unit': '', 'description': 'GAT', 'parent': None, 'relation': None}, {'admission_criteria': 74

In [9]:
final_data_to_json = {f'{p}-{m}': criterias[(p,m)]['data'] for p,m in program_major_codes}

In [11]:
filename = 'export-to-portal.json'

with open(filename,'w') as f:
    print(json.dumps(final_data_to_json, indent=1),
          file=f)

In [12]:
print(json.dumps(final_data_to_json, indent=1))

{
 "10020118620101A-A": {
  "major_cupt_code": {
   "program_code": "10020118620101A",
   "program_type": "\u0e20\u0e32\u0e29\u0e32\u0e44\u0e17\u0e22 \u0e1b\u0e01\u0e15\u0e34",
   "program_type_code": "A",
   "faculty": 30,
   "major_code": "A",
   "title": "\u0e1a\u0e18.\u0e1a. \u0e2a\u0e32\u0e02\u0e32\u0e27\u0e34\u0e0a\u0e32\u0e01\u0e32\u0e23\u0e08\u0e31\u0e14\u0e01\u0e32\u0e23\u0e2d\u0e38\u0e15\u0e2a\u0e32\u0e2b\u0e01\u0e23\u0e23\u0e21\u0e01\u0e32\u0e23\u0e1a\u0e23\u0e34\u0e01\u0e32\u0e23",
   "major_title": "\u0e27\u0e34\u0e0a\u0e32\u0e40\u0e2d\u0e01\u0e01\u0e32\u0e23\u0e08\u0e31\u0e14\u0e01\u0e32\u0e23\u0e42\u0e23\u0e07\u0e41\u0e23\u0e21",
   "component_weight_type": ""
  },
  "criterias": {
   "27": [
    {
     "slots": 15,
     "admission_criteria": {
      "admission_criteria": {
       "admission_project": 27,
       "faculty": 30,
       "version": 2,
       "is_deleted": false,
       "created_at": "2021-02-01T17:01:31.999",
       "created_by": "regis",
       "additional_